# Evaluation

Under this scenario, a score from 0-10, with 10 being closest match, is generated comparing a baseline response against an actual response. This scenario could be very useful to check RAG responses.

### Import the required packages

In [1]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.14"
#r "nuget: Scriban"
#r "nuget: dotenv.net"

using Azure;
using Azure.AI.OpenAI;
using Scriban;

using dotenv.net;
DotEnv.Load();

var endpoint=Environment.GetEnvironmentVariable("OPENAI_URI");
var key=Environment.GetEnvironmentVariable("OPENAI_KEY");
var model=Environment.GetEnvironmentVariable("OPENAI_GPT_DEPLOYMENT");

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.14 dotenv.net, 3.1.3 Scriban, 5.9.1

### Process a completion using the Azure SDK

In [2]:
async Task<string> ProcessCompletion(string prompt, int max_tokens=500, float temperature=0.3f)
{
    OpenAIClient client = new(new Uri(endpoint), new AzureKeyCredential(key));

    var chatCompletionsOptions = new ChatCompletionsOptions()
    {
        DeploymentName = model, //This must match the custom deployment name you chose for your model
        Messages =
        {
            new ChatRequestAssistantMessage(prompt),
        },
        MaxTokens = max_tokens,
        Temperature = temperature
    };

    Response<ChatCompletions> response = await client.GetChatCompletionsAsync(chatCompletionsOptions);

    return response.Value.Choices[0].Message.Content;
}

### Render an Scriban template

In [3]:
string RenderTemplate(string template, dynamic data) {
    var templateObject = Template.Parse(template);
    return templateObject.Render(data);
}

### Prepare the Prompt

In [8]:
var promptTemplate = @"On a scale from 0-10 with 10 being a very good match, how close of a match is the Actual Response to the Baseline Response:

Baseline response:
{{reference}}

Actual Response:
{{actual}}

Output format:
{
""score"":0,
""reason"":""""
}

Output in JSON format only.";
promptTemplate

On a scale from 0-10 with 10 being a very good match, how close of a match is the Actual Response to the Baseline Response:

Baseline response:
{{reference}}

Actual Response:
{{actual}}

Output format:
{
"score":0,
"reason":""
}

Output in JSON format only.

### Prepare the comparison

In [9]:
var EXPECTED_ANSWER = "The speed of light in a vacuum is approximately 299,792,458 meters per second (m/s). To convert this value to kilometers per second (km/s), we divide by 1,000 since there are 1,000 meters in a kilometer. Therefore, the speed of light in kilometers per second is approximately 299,792.458 km/s.";
var ANSWER = "The speed of light is approximately 300,000 km/s.";
var data = new {EXPECTED_ANSWER=EXPECTED_ANSWER, ANSWER=ANSWER};
var prompt = RenderTemplate(promptTemplate, new {reference=EXPECTED_ANSWER, actual=ANSWER});
prompt

On a scale from 0-10 with 10 being a very good match, how close of a match is the Actual Response to the Baseline Response:

Baseline response:
The speed of light in a vacuum is approximately 299,792,458 meters per second (m/s). To convert this value to kilometers per second (km/s), we divide by 1,000 since there are 1,000 meters in a kilometer. Therefore, the speed of light in kilometers per second is approximately 299,792.458 km/s.

Actual Response:
The speed of light is approximately 300,000 km/s.

Output format:
{
"score":0,
"reason":""
}

Output in JSON format only.

### Process the completion

In [10]:
Console.WriteLine(await ProcessCompletion(prompt));

{
"score":9,
"reason":"The Actual Response is a rounded approximation of the Baseline Response, which is accurate and conveys the same information in a simplified form."
}
